# Areios Pagos Crawling

In [7]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from pprint import pprint

import time

In [4]:
def open_firefox():
        firefox_options = Options()
        driver_path = r'C:\Program Files\geckodriver-v0.35.0-win64\geckodriver.exe'
        service = Service(driver_path)
        driver = webdriver.Firefox(service=service, options=firefox_options)
        return driver

In [ ]:
def search_decisions(driver, year="2024"):
        driver.get("https://areiospagos.gr/nomologia/apofaseis.asp")
        wait = WebDriverWait(driver, 10)

        # fill 2024 as the year to be searched
        year_input = wait.until(expected_conditions.presence_of_element_located((By.NAME, 'x_ETOS')))
        year_input.clear()
        year_input.send_keys(year)

        # select 'ΟΛΕΣ' stis 'Αποφάσεις'
        select_tmhma = Select(driver.find_element(By.NAME, 'X_TMHMA'))
        select_tmhma.select_by_value('6')

        # select 'ΟΛΕΣ' sto 'Τμήμα'
        select_sub_tmhma = Select(driver.find_element(By.NAME, 'X_SUB_TMHMA'))
        select_sub_tmhma.select_by_value('1')

        search_button = driver.find_element(By.NAME, 'submit_krit')
        search_button.click()

        time.sleep(2)
        print(driver.page_source)
        return driver.page_source

# Usage
driver = open_firefox()
html_content = search_decisions(driver, "2024") 

In [ ]:
def extract_decision_links(html_content):
        soup = BeautifulSoup(html_content, 'html.parser')
        links = []

        for a_tag in soup.select('a.blue10_cursor[href^="apofaseis_DISPLAY.asp"]'):
                decision_id = a_tag.text.strip()
                info = a_tag.get('href').split('info')[1] if 'info=' in a_tag.get('href') else ""

                href = a_tag.get('href')

                url = 'full_url = f"https://areiospagos.gr/nomologia/{href}"'

                links.append({
                        'decision_id': decision_id,
                        'info': info,
                        'url': url
                })

        return links

links = extract_decision_links(html_content)
pprint(links)       